In [1]:
from   scipy  import stats
import pandas as pd
import numpy  as np

In [2]:
dataset = pd.read_csv('HRV(RMSSD) Data Set.csv')
dataset.columns = ['id', 'group'] + list(range(0, 21, 2))

# mean values per phase
groups  = dataset.copy()

groups['T1'] = groups.iloc[:,3:8].mean(axis = 1)   # [ 2, 10]
groups['T2'] = groups.iloc[:,4:8].mean(axis = 1)   # [ 4, 10]
groups['T3'] = groups.iloc[:,5:8].mean(axis = 1)   # [ 6, 10]
groups['T4'] = groups.iloc[:,6:8].mean(axis = 1)   # [ 8, 10]
groups['T5'] = groups.iloc[:,7]                    # [10, 10]
groups['T6'] = groups.iloc[:,8:10].mean(axis = 1)  # [12, 14]
groups['T7'] = groups.iloc[:,10:13].mean(axis = 1) # [14, 20]

groups  = groups.iloc[:,[0, 1, 2, -7, -6, -5, -4, -3, -2, -1]].copy()

# mean times ([2, 10], [12, 14], [16, 20])
groups.columns = ['id', 'group', 0, ' 2-10', ' 4-10', ' 6-10', ' 8-10', '10-10', '12-14', '16-18']

display(dataset.head())
display(groups.head())

,id,group,0,2,4,6,8,10,12,14,16,18,20
0,1,Group A,0,4.349499,2.332740,0.866309,1.128269,2.323326,6.772574,10.429308,4.126795,3.542454,0.132981
1,5,Group A,0,1.677594,0.039062,-1.805596,-0.699485,4.100669,5.306848,7.757402,4.743457,1.978138,-0.987276
2,7,Group A,0,4.602275,5.052061,3.931895,6.457339,5.521690,5.833740,9.396320,8.283913,1.771258,-1.243988
3,9,Group A,0,-8.810611,-6.831975,-4.594777,-4.415607,1.209170,4.630715,9.268901,11.916559,13.161639,16.832487
4,10,Group A,0,-16.864643,-8.846720,-2.649177,-2.040748,0.566561,0.053469,-4.269396,-2.101262,6.197839,9.532302


,id,group,0,2-10,4-10,6-10,8-10,10-10,12-14,16-18
0,1,Group A,0,2.200029,1.662661,1.439301,1.725797,2.323326,8.600941,2.600743
1,5,Group A,0,0.662449,0.408662,0.531863,1.700592,4.100669,6.532125,1.911439
2,7,Group A,0,5.113052,5.240746,5.303641,5.989514,5.521690,7.615030,2.937061
3,9,Group A,0,-4.688760,-3.658297,-2.600404,-1.603218,1.209170,6.949808,13.970229
4,10,Group A,0,-5.966946,-3.242521,-1.374455,-0.737094,0.566561,-2.107963,4.542960


In [3]:
def categorical_individual_design(df):
    """ creates the design matrix and target vector if assumed each time step was independent"""
    m = df.shape[1] - 3
    n = len(df)
    X = np.zeros((n * m, m * 2))
    y = np.zeros((n * m))
    for i in range(n):
        A = df.iloc[i, 1] == 'Group A'
        for j in range(m):
            X[i * m + j, j] = 1
            X[i * m + j, m + j] = A
            y[i * m + j] = df.iloc[i, 3 + j]
    return X, y

def categorical_shared_design(df):
    """ creates the design matrix and target vector if assumed each time step was independent"""
    m = df.shape[1] - 3
    n = len(df)
    X = np.zeros((n * m, m))
    y = np.zeros((n * m))
    for i in range(n):
        A = df.iloc[i, 1] == 'Group A'
        for j in range(m):
            X[i * m + j, j] = A
            y[i * m + j] = df.iloc[i, 3 + j]
    X = np.insert(X, 0, 1, 1)
    return X, y

def regression_design(df):
    """ creates the design matrix and target vector if assumed time is linear """
    m = df.shape[1] - 3
    n = len(df)
    X = np.zeros((n * m, 2))
    y = np.zeros((n * m))
    for i in range(n):
        A = df.iloc[i,1] == 'Group A'
        for j in range(m):
            if isinstance(df.columns[3 + j], int):
                X[i * m + j] = A, df.columns[3 + j]
            else:
                num = (int(df.columns[3 + j][:2]) + int(df.columns[3 + j][-2:])) / 2
                X[i * m + j] = A, num
            y[i * m + j] = df.iloc[i, 3 + j]
    X = np.insert(X, 0, 1, 1)
    return X, y

def compute_statistics(X, y):
    """ computes t-statistics and significances """
    theta = np.linalg.solve(X.T @ X, X.T @ y)
    m     = len(theta)
    r     = X @ theta - y
    s     = np.std(r, ddof = m)
    sed   = s * np.sqrt(np.diag(np.linalg.inv(X.T @ X)))
    alpha = 0.05
    t     = (theta - 0) / sed
    T     = stats.t(len(X) - m)
    c     = T.ppf(1 - alpha / 2) * sed
    ci    = np.array([theta, theta - c, theta + c])
    return t, 1 - T.cdf(t), ci

def categorical_shared_baseline(df):
    X, y     = categorical_shared_design(df)
    t, p, ci = compute_statistics(X, y)
    m        = len(t)
    # ignore statistics for intercepts
    return t[1:], p[1:], ci[:,1:]

def categorical_individual_baseline(df):
    X, y     = categorical_individual_design(df)
    t, p, ci = compute_statistics(X, y)
    m        = len(t)
    # ignore statistics for intercepts
    return t[m // 2:], p[m // 2:], ci[:,m // 2:]

def regression(df):
    X, y     = regression_design(df)
    t, p, ci = compute_statistics(X, y) # [intercept, group, time]
    # inspect the statistics only for the group parameter
    return t[1], p[1], ci[:,1]

In [4]:
def pprint(t, p, ci):
    if isinstance(t, np.ndarray):
        try:
            rows = zip([f'theta[{column:2d}]   ' for column in df.columns[3:]], t, p, ci.T)
        except:
            rows = zip([f'theta[{column:5s}]' for column in df.columns[3:]], t, p, ci.T)
    else:
        rows     = [('theta[t]    ', t, p, ci.T)]
    print('                    |  value      |  c. interval (0.95)         |  t-statistic  |  p-value   |  significance')
    print('      --------------+-------------+-----------------------------+---------------+------------+--------------')
    for g, t, p, ci in rows:
        if p < 0.001:
            s = '***'
        elif p < 0.01:
            s = '**'
        elif p < 0.05:
            s = '*'
        else:
            s = ''
        print(f'      {g}  |  {ci[0]:9.6f}  |  [{ci[1]:10.6f}, {ci[2]:10.6f}]   |  {t:.6f}     |  {p:.6f}  |  {s:6s}')
    print()

def bold(string):
    return f'\033[1m{string}\033[0m'

def gen_legend(eq):
    legend = []
    if 'y[t]' in eq:
        legend.append('y[t]         : response vector at time t')
    if 'y[t1-t2]' in eq:
        legend.append('y[t1-t2]     : response vector with mean value between time t1 and t2')
    if 'b[t]' in eq:
        legend.append('b[t]         : baseline value at time t')
    if 'b[t1-t2]' in eq:
        legend.append('b[t1-t2]     : baseline value between time t1 and t2')
    if 'b ' in eq:
        legend.append('b            : baseline value for all timesteps')
    if 'g[A]' in eq:
        legend.append('g[A]         : dummy variable where 1 if in group A else 0')
    if 'theta[A]' in eq:
        legend.append('theta[A]     : how being in Group A affects y')
    if 'theta[t]' in eq:
        legend.append('theta[t]     : how time affects y categorically')
    if 'theta[t1-t2]' in eq:
        legend.append('theta[t1-t2] : how time period affects y categorically between time t1 and t2')
    if 'theta ' in eq:
        legend.append('theta        : how time affects y linearly')
    return f'      • ' + '\n      • '.join(legend) + '\n'

titles  = 'categorical time (individual baseline)', 'categorical time (shared baseline)', 'regress on time (linear in time)'
eqs     = 'y[t] = b[t] + g[A]·theta[t]', 'y[t] = b + g[A]·theta[t]', 'y[t] = b + g[A]·theta[A] + t·theta '

funcs   = categorical_individual_baseline, categorical_shared_baseline, regression

for i, (df, name) in enumerate(zip([dataset, groups], ['individual time', 'group-by-phase time (mean)'])):
    print(bold(name), end = '\n\n')

    for (title, eq, func) in zip(titles, eqs, funcs):
        if i:
            eq = eq.replace('[t]', '[t1-t2]')
        print(f'    {title}\n')
        print(f'        {bold(eq)}\n')
        print(gen_legend(eq))
        pprint(*func(df))


individual time

    categorical time (individual baseline)

        y[t] = b[t] + g[A]·theta[t]

      • y[t]         : response vector at time t
      • b[t]         : baseline value at time t
      • g[A]         : dummy variable where 1 if in group A else 0
      • theta[t]     : how time affects y categorically

                    |  value      |  c. interval (0.95)         |  t-statistic  |  p-value   |  significance
      --------------+-------------+-----------------------------+---------------+------------+--------------
      theta[ 2]     |   2.198571  |  [ -7.813509,  12.210651]   |  0.431416     |  0.333174  |        
      theta[ 4]     |   3.989524  |  [ -6.022556,  14.001604]   |  0.782847     |  0.217040  |        
      theta[ 6]     |   6.960178  |  [ -3.051902,  16.972258]   |  1.365765     |  0.086307  |        
      theta[ 8]     |   8.899076  |  [ -1.113005,  18.911156]   |  1.746226     |  0.040687  |  *     
      theta[10]     |   9.494219  |  [ -0.517862,  